In [35]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, GridSearchCV

import lightgbm as lgb

In [4]:
folder = "without_na"

train = pd.read_csv("{}/train_clean.csv".format(folder))
val = pd.read_csv("{}/validation_clean.csv".format(folder))

In [32]:
# divide data into 2 groups: clean and 'notclean'
    
clean_df = train[train['clean'] == 1].drop('clean', axis=1)
notclean_df = train[train['clean'] == 0].drop('clean', axis=1)

val_clean_df = val[val['clean'] == 1].drop('clean', axis=1)
val_not_clean_df = val[val['clean'] == 0].drop('clean', axis=1)

In [60]:
def get_class_weights(labels):
    classes, class_counts = np.unique(labels, return_counts=True)
    weights = 1 - class_counts / np.sum(class_counts)
    return {c:w for c in classes for w in weights}

In [33]:
# prepare datasets
features_to_drop = ['objid', 'class']

X_train = clean_df.drop(features_to_drop, axis=1).values
y_train = clean_df['class'].values

X_test = val_clean_df.drop(features_to_drop, axis=1).values
y_test = val_clean_df['class'].values

In [41]:
params_grid = {
    'learning_rate': [0.05, 0.1, 0.3],
    'boosting': ['dart', 'gbdt'],
    'num_leaves': [30, 60, 90],
    'num_boost_round': [400]
}

gbm_for_clean = lgb.LGBMClassifier(objective='multiclass',
                                  num_class = 3)

kf = KFold(n_splits=3, random_state=41, shuffle=True)
gridSearch = GridSearchCV(gbm_for_clean, params_grid, cv=kf)

gridSearch.fit(X_train, y_train)

C:\Users\alina.bogdanova\anaconda3\lib\site-packages\lightgbm\engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\lightgbm\engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{

C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\lightgbm\engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in f

C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\lightgbm\engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in f

C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\lightgbm\engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\alina.bogdanova\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in f

KeyboardInterrupt: 

In [40]:
print(gridSearch.best_params_)
print(gridSearch.best_score_)

{'boosting': 'dart', 'learning_rate': 0.5, 'num_leaves': 90}
0.8756723963599595


In [65]:
params = {
    'learning_rate': 0.1,
    'num_leaves': 90,
    'boosting': 'dart'
}

gbm_for_clean = lgb.LGBMClassifier(objective='multiclass',
                                  num_class = 3,
                                  class_weight=get_class_weights(y_test))
gbm_for_clean.fit(X_train, y_train,
                 eval_set=[(X_test, y_test)],
                 early_stopping_rounds=50)
                  

[1]	valid_0's multi_logloss: 1.00302
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 0.923869
[3]	valid_0's multi_logloss: 0.856388
[4]	valid_0's multi_logloss: 0.798321
[5]	valid_0's multi_logloss: 0.748019
[6]	valid_0's multi_logloss: 0.704083
[7]	valid_0's multi_logloss: 0.665969
[8]	valid_0's multi_logloss: 0.634078
[9]	valid_0's multi_logloss: 0.605276
[10]	valid_0's multi_logloss: 0.580448
[11]	valid_0's multi_logloss: 0.557714
[12]	valid_0's multi_logloss: 0.538136
[13]	valid_0's multi_logloss: 0.518773
[14]	valid_0's multi_logloss: 0.501804
[15]	valid_0's multi_logloss: 0.486836
[16]	valid_0's multi_logloss: 0.474326
[17]	valid_0's multi_logloss: 0.461966
[18]	valid_0's multi_logloss: 0.451611
[19]	valid_0's multi_logloss: 0.441897
[20]	valid_0's multi_logloss: 0.433808
[21]	valid_0's multi_logloss: 0.425915
[22]	valid_0's multi_logloss: 0.418567
[23]	valid_0's multi_logloss: 0.41288
[24]	valid_0's multi_logloss: 0.407393
[25]	valid_0'

LGBMClassifier(boosting_type='gbdt',
        class_weight={0: 0.7366275546741585, 1: 0.7366275546741585, 2: 0.7366275546741585},
        colsample_bytree=1.0, learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_class=3, num_leaves=31,
        objective='multiclass', random_state=None, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)